In [8]:
import requests
from dotenv import load_dotenv
import os
import requests
import pandas as pd
import time
from datetime import datetime, timedelta, timezone


# Load the variables from .env
load_dotenv()

# Access your API key
api_key = os.getenv("api_key")

city = "New York"
url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"

response = requests.get(url)
data = response.json() 

print(data)

{'coord': {'lon': -74.006, 'lat': 40.7143}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01d'}], 'base': 'stations', 'main': {'temp': 11.43, 'feels_like': 10.43, 'temp_min': 10.32, 'temp_max': 12.28, 'pressure': 1015, 'humidity': 69, 'sea_level': 1015, 'grnd_level': 1013}, 'visibility': 10000, 'wind': {'speed': 6.17, 'deg': 240}, 'clouds': {'all': 0}, 'dt': 1763752082, 'sys': {'type': 1, 'id': 4610, 'country': 'US', 'sunrise': 1763725814, 'sunset': 1763760842}, 'timezone': -18000, 'id': 5128581, 'name': 'New York', 'cod': 200}


In [14]:


LOCATIONS = {
    "Park City": (40.6461, -111.4980),
    "Big Cottonwood": (40.6094, -111.7102)
}

# Pull 1 year back
END_DATE = datetime.now(timezone.utc)
START_DATE = END_DATE - timedelta(days=365)

# ----------------------------
# HELPERS
# ----------------------------

def unix_ts(dt):
    """Convert datetime -> Unix timestamp."""
    return int(dt.timestamp())


def get_day(lat, lon, date):
    """Retrieve one full day of hourly weather."""
    start_ts = unix_ts(datetime(date.year, date.month, date.day, 0, 0))
    end_ts   = unix_ts(datetime(date.year, date.month, date.day, 23, 59))

    url = (
        "https://history.openweathermap.org/data/2.5/history/city"
        f"?lat={lat}&lon={lon}"
        f"&type=hour&start={start_ts}&end={end_ts}"
        f"&appid={api_key}&units=metric"
    )

    r = requests.get(url)

    if r.status_code != 200:
        print(f"[ERROR] {r.status_code} {r.text}")
        return None

    return r.json()


def parse_day(json_data, location_name):
    """Extract hourly rows from the API response."""
    if not json_data or "list" not in json_data:
        return []

    rows = []

    for hour in json_data["list"]:
        weather = hour.get("weather", [{}])[0]

        rows.append({
            "location": location_name,
            "timestamp": hour.get("dt"),
            "temp": hour.get("main", {}).get("temp"),
            "feels_like": hour.get("main", {}).get("feels_like"),
            "pressure": hour.get("main", {}).get("pressure"),
            "humidity": hour.get("main", {}).get("humidity"),
            "wind_speed": hour.get("wind", {}).get("speed"),
            "clouds": hour.get("clouds", {}).get("all"),
            "weather_main": weather.get("main"),
            "weather_description": weather.get("description")
        })

    return rows

def collect_weather():
    all_rows = []

    for name, (lat, lon) in LOCATIONS.items():
        print(f"\nCollecting: {name}")

        # Iterate 365 days
        date = START_DATE
        while date <= END_DATE:
            day_json = get_day(lat, lon, date)

            if day_json:
                rows = parse_day(day_json, name)
                all_rows.extend(rows)

            time.sleep(1)  # rate limit

            date += timedelta(days=1)

    return pd.DataFrame(all_rows)


# ----------------------------
# RUN
# ----------------------------

if __name__ == "__main__":
    print("Fetching historical weather...")
    df = collect_weather()

    print("\nSample:")
    print(df.head())

    df.to_csv("ski_weather_last_year.csv", index=False)
    print("\nSaved to ski_weather_last_year.csv")


Fetching historical weather...

Collecting: Park City
[ERROR] 400 {"code":400000,"message":"requested data is out of allowed range"}



KeyboardInterrupt: 